<a href="https://colab.research.google.com/github/analyticsariel/market-research-data/blob/main/037_Get_Comparables_RealtyMole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps
### Get Keys
1) Get Free [Rapid API Key by creating an account](https://rapidapi.com/marketplace)

Store each key in a central and private location.

### Suscribe to APIs

1) Suscribe to [Realty Mole on Rapid API](https://rapidapi.com/realtymole/api/realty-mole-property-api/endpoints)

## <font color="blue">Imports</font>

In [ ]:
# working with files
from google.colab import drive, files
# api post/get
import requests
from bs4 import BeautifulSoup
# data manipulation
import json
import pandas as pd
import numpy as np
# plotting
import plotly.express as px

## <font color="blue">Functions</font>

### Realty Mole

In [ ]:
def realty_mole_sale_price(rapid_api_key,
                           street,
                           city,
                           state,
                           squareFootage=None,
                           bathrooms=None,
                           bedrooms=None,
                           propertyType=None,
                           comp_count=5):
    """
    This endpoint retrieves the property's 
    sales price estimate by a comp analysis from Realty Mole's.
    
    The Find Property API Web Service is located at: 
    https://rapidapi.com/realtymole/api/realty-mole-property-api/endpoints

    Parameters
    ----------
    @rapid_api_key [string]: Key to access data from Rapid API
    @address [string]: The address of the property to search
    @city [string]: The city of the property to search
    @state [string]: The state of the property to search
    @squareFootage [string]: The square footage of the property to search
    @bathrooms [string]: The bathrooms of the property to search
    @comp_count [string]: The number of comparable properties to evaluate sales price

    Returns
    -------
    [json] API response

    """
    # get address
    address = "{0}, {1}, {2}".format(str(street), 
                                     str(city), 
                                     str(state))
    
    # api url
    url = "https://realty-mole-property-api.p.rapidapi.com/salePrice"

    # payload
    querystring = {"compCount":str(comp_count),
                   "address":address,
                   "squareFootage":squareFootage,
                   "bathrooms":bathrooms,
                   "bedrooms":bedrooms,
                   "propertyType":propertyType}

    # header
    headers = {
        'x-rapidapi-key': rapid_api_key,
        'x-rapidapi-host': "realty-mole-property-api.p.rapidapi.com"
        }

    # response
    response = requests.request("GET", url, headers=headers, params=querystring)
    
    # check if there is an error on the request
    if response.status_code == 200: # no error
        return response
    else:
        return """Error {0}. See API source documentation for message and error code - https://rapidapi.com/realtymole/api/realty-mole-property-api/endpoints"""\
        .format(response.status_code)

## <font color="blue">Local & Constants</font>

In [ ]:
# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/'

Mounted at /content/drive


## <font color="blue">Data</font>

### Get API Keys

In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key
mapbox_key = df_api_keys.loc[df_api_keys['API'] =='mapbox']['KEY'].iloc[0] # replace this with your own key

## <font color="blue">Test Address</font>
[122 George St, South Amboy, NJ 08879](https://www.zillow.com/homedetails/122-George-St-South-Amboy-NJ-08879/39166231_zpid/)

In [ ]:
street = "728 June Lake Ln"
city = "Brandon"
state = "FL"
zip_code = "33510"

## <font color="green">Realty Mole Comparables</font>
<font color="gray">*Search Realty Mole comparables*</font>

[Rapid API - Realty Mole Endpoints](https://rapidapi.com/realtymole/api/realty-mole-property-api/endpoints)

[Realty Mole Website](https://www.realtymole.com/home)

In [ ]:
# get api response
realty_mole_sales_response = realty_mole_sale_price(rapid_api_key=rapid_api_key,
                                                    street=street,
                                                    city=city,
                                                    state=state,
                                                    propertyType="Single Family",
                                                    comp_count=5)

In [ ]:
realty_mole_sales_response.json()

{'latitude': 27.9581312,
 'listings': [{'address': '727 June Lake Ln',
   'bathrooms': 2,
   'bedrooms': 4,
   'city': 'Brandon',
   'correlation': 0.998,
   'county': 'Hillsborough County',
   'daysOld': 12.33,
   'distance': 0.05431986622029869,
   'formattedAddress': '727 June Lake Ln, Brandon, FL 33510',
   'id': '727-June-Lake-Ln,-Brandon,-FL-33510',
   'latitude': 27.958617,
   'longitude': -82.296714,
   'price': 355000,
   'propertyType': 'Single Family',
   'publishedDate': '2021-03-06T14:43:18.467Z',
   'squareFootage': 2307,
   'state': 'FL',
   'zipcode': '33510'},
  {'address': '610 June Lake Ln',
   'bathrooms': 2,
   'bedrooms': 3,
   'city': 'Brandon',
   'correlation': 0.9942,
   'county': 'Hillsborough County',
   'daysOld': 80.26,
   'distance': 0.14998801188978036,
   'formattedAddress': '610 June Lake Ln, Brandon, FL 33510',
   'id': '610-June-Lake-Ln,-Brandon,-FL-33510',
   'latitude': 27.959437,
   'longitude': -82.297038,
   'price': 319700,
   'propertyType': '

In [ ]:
# transform to dataframe
df_realty_comps = pd.DataFrame(realty_mole_sales_response.json()['listings'])
# view contents
df_realty_comps

,id,formattedAddress,longitude,latitude,city,state,zipcode,price,publishedDate,distance,daysOld,correlation,address,county,bedrooms,bathrooms,propertyType,squareFootage
0,"727-June-Lake-Ln,-Brandon,-FL-33510","727 June Lake Ln, Brandon, FL 33510",-82.296714,27.958617,Brandon,FL,33510,355000,2021-03-06T14:43:18.467Z,0.054320,12.33,0.9980,727 June Lake Ln,Hillsborough County,4,2,Single Family,2307
1,"610-June-Lake-Ln,-Brandon,-FL-33510","610 June Lake Ln, Brandon, FL 33510",-82.297038,27.959437,Brandon,FL,33510,319700,2020-12-28T16:18:33.023Z,0.149988,80.26,0.9942,610 June Lake Ln,Hillsborough County,3,2,Single Family,2137
2,"819-Windsor-Cir,-Brandon,-FL-33510","819 Windsor Cir, Brandon, FL 33510",-82.298883,27.958021,Brandon,FL,33510,220000,2020-11-12T16:01:07.444Z,0.218728,126.27,0.9915,819 Windsor Cir,Hillsborough County,3,2,Single Family,1485
3,"1207-Windy-Hill-Dr,-Brandon,-FL-33510","1207 Windy Hill Dr, Brandon, FL 33510",-82.299037,27.959028,Brandon,FL,33510,299900,2021-03-06T14:28:13.598Z,0.253969,12.34,0.9911,1207 Windy Hill Dr,Hillsborough County,4,2,Single Family,1849
4,"1203-Windsor-Cir,-Brandon,-FL-33510","1203 Windsor Cir, Brandon, FL 33510",-82.300393,27.958024,Brandon,FL,33510,237900,2020-12-12T14:18:50.017Z,0.367052,96.34,0.9866,1203 Windsor Cir,Hillsborough County,3,2,Single Family,1400


In [ ]:
# view in chart
px.set_mapbox_access_token(mapbox_key)
fig = px.scatter_mapbox(df_realty_comps, 
                        lat="latitude", 
                        lon="longitude",     
                        color="address", 
                        color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=30, 
                        zoom=15,
                        title="Comparables for {0}".format(street))
fig.show()

# End Notebook